В этом коде я создаю текстовый csv файл следующейго содержания:

данные из html файлов отдела CMK в виде текстовой базы данных
Вид:

целевые столбцы Query, Answer, Department, Link, где

Query = названию файла html (по возможности без текстового мусора)

Answer содержит всю содержательную часть файлов папки CMK, соответствующую Query в этой строке. Табличные данные вида вопрос-ответ были распарсены немного по-другому, строки таких таблиц стали отдельными строками в нашем csv, в этом случае создано несколько строк с одинаковым Query, соответствуюшим названию файла (Напрмер, Частые Вопросы), а внутри Answer будет лишь один смысловой вопрос-ответ, как отдельная смысловая еденица.

Department - отдел (=CMK)

Link - ссылка на конфлюенс

Загрузка необходимых библиотек. BeautifulSoup помогает распарсить данные из html файлов, os помогает работать с файлами, папками, переименованием, pandas - библиотека для обработки и анализа данных, fuzzywuzzy используется для нахождения наиболее похожих строк, может использоваться для нахождения опечаток

In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
!pip install fuzzywuzzy[speedup]
from fuzzywuzzy import process

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Задаю папку с файлами CMK

In [3]:
directory = '/content/drive/MyDrive/PEK/СМК'

Инициализирую пустой список для распарсенных смысловых абзацев

In [4]:
paragraphs_dataframes = []

Функиця, которая складывает данные из тегов /p и /td (со значимой информацией для этой папки) в распарсенные смысловые абзаци, а потом склеивает их пофайлово в один дата фрейм

In [5]:
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)

    # Check if the file exists and is an HTML file
    if os.path.isfile(file_path) and filename.endswith('.html'):
        # Load the HTML file
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract all paragraphs and concatenate their text
        paragraphs = soup.find_all('p')
        tables = soup.find_all('td')
        paragraphs_text = ' '.join(p.get_text(strip=True) for p in paragraphs)
        table_items_text = ' '.join(table.get_text(strip=True) for table in tables)
        combined_text = paragraphs_text + ' ' + table_items_text

        # Create a DataFrame for the concatenated paragraphs
        df_paragraphs = pd.DataFrame([[os.path.splitext(filename)[0], combined_text]],
                                     columns=['Query', 'Answer'])

        # Append the DataFrame to the list
        paragraphs_dataframes.append(df_paragraphs)

In [6]:
combined_paragraphs_df = []
combined_paragraphs_df = pd.concat(paragraphs_dataframes, ignore_index=True)

In [7]:
display(combined_paragraphs_df)

,Query,Answer
0,Технологическая инструкция _Проверка данных ко...,АКТУАЛЬНО Статус конфиденциальности Для внутре...
1,Технологическая инструкция «Правила подписания...,АКТУАЛЬНО Статус конфиденциальности Для внутре...
2,Технологическая инструкция _Архивация документ...,АКТУАЛЬНО Статус конфиденциальности Для внутре...
3,Инструкция пользователя «Оформление груза в СП...,"АКТУАЛЬНО Взамен: Версия 1.0, ИП-УКС-003 Инст..."
4,Инструкция пользователя «Создание нового Контр...,АКТУАЛЬНО (Взамен: ИП-УКС-001 Инструкции польз...
...,...,...
107,GLOBAL - Продуктовая книга - База знаний,Информация на сайте по перевозке в Армению...
108,Внутрикорпоративные термины - Продуктовая книг...,"Доставка товара в определенные дату и время, в..."
109,Концепт ролевой модели прав в новом ЛК,Права полномочий пользователя ЛК (свойства лог...
110,Программа лояльности,Регистрация в программе лояльности в офисе Ч...


Удаляю типичный мусор из начала распарсенных файлов

In [8]:
def clean_content(content):
    if "Для внутреннего использования Область регламентации" in content:
        return content.split("Для внутреннего использования Область регламентации", 1)[1]
    return content

In [9]:
combined_paragraphs_df['Answer'] = combined_paragraphs_df['Answer'].apply(clean_content)
display(combined_paragraphs_df)

,Query,Answer
0,Технологическая инструкция _Проверка данных ко...,Требования к описанию операции «Проверка данн...
1,Технологическая инструкция «Правила подписания...,Описание правил подписания ПЭ/ЭР Ответственны...
2,Технологическая инструкция _Архивация документ...,Установление единых правил архивации документ...
3,Инструкция пользователя «Оформление груза в СП...,Описание операции по оформлению груза к перев...
4,Инструкция пользователя «Создание нового Контр...,Требования к описанию операции «Создание ново...
...,...,...
107,GLOBAL - Продуктовая книга - База знаний,Информация на сайте по перевозке в Армению...
108,Внутрикорпоративные термины - Продуктовая книг...,"Доставка товара в определенные дату и время, в..."
109,Концепт ролевой модели прав в новом ЛК,Права полномочий пользователя ЛК (свойства лог...
110,Программа лояльности,Регистрация в программе лояльности в офисе Ч...


Удаляю типичный мусор из конца распарсенных файлов

In [10]:
unwanted_text = 'Добавить комментарий ОЦЕНОЧНАЯ ЛИЦЕНЗИЯВам нравится Confluence? Пожалуйста, рассмотритедля приобретения егосегодня.'

# Function to remove unwanted text from the end of each cell, if present
def remove_unwanted_text(content):
    if unwanted_text in content:
        return content.replace(unwanted_text, '').strip()
    return content

In [11]:
combined_paragraphs_df['Answer'] = combined_paragraphs_df['Answer'].apply(remove_unwanted_text)
display(combined_paragraphs_df)

,Query,Answer
0,Технологическая инструкция _Проверка данных ко...,Требования к описанию операции «Проверка данны...
1,Технологическая инструкция «Правила подписания...,Описание правил подписания ПЭ/ЭР Ответственный...
2,Технологическая инструкция _Архивация документ...,Установление единых правил архивации документо...
3,Инструкция пользователя «Оформление груза в СП...,Описание операции по оформлению груза к перево...
4,Инструкция пользователя «Создание нового Контр...,Требования к описанию операции «Создание новог...
...,...,...
107,GLOBAL - Продуктовая книга - База знаний,Информация на сайте по перевозке в Армению...
108,Внутрикорпоративные термины - Продуктовая книг...,"Доставка товара в определенные дату и время, в..."
109,Концепт ролевой модели прав в новом ЛК,Права полномочий пользователя ЛК (свойства лог...
110,Программа лояльности,Регистрация в программе лояльности в офисе Ч...


Убираю артефакты из названия файлов, чтобы модель, которая после будет мэтчить их со своими названиями файлов (очень похожими, но не точно такими же) работала быстрее и эффективнее

In [12]:
# Texts to remove
unwanted_endings = [
    ' - CMK - База знаний',
    ' - База знаний'
]

# Function to remove unwanted text endings from 'File' column
def remove_unwanted_endings(file_name):
    for ending in unwanted_endings:
        if file_name.endswith(ending):
            return file_name[:-len(ending)].strip()
    return file_name

In [13]:
combined_paragraphs_df['Query'] = combined_paragraphs_df['Query'].apply(remove_unwanted_endings)

In [14]:
display(combined_paragraphs_df)

,Query,Answer
0,Технологическая инструкция _Проверка данных ко...,Требования к описанию операции «Проверка данны...
1,Технологическая инструкция «Правила подписания...,Описание правил подписания ПЭ/ЭР Ответственный...
2,Технологическая инструкция _Архивация документ...,Установление единых правил архивации документо...
3,Инструкция пользователя «Оформление груза в СПО»,Описание операции по оформлению груза к перево...
4,Инструкция пользователя «Создание нового Контр...,Требования к описанию операции «Создание новог...
...,...,...
107,GLOBAL - Продуктовая книга,Информация на сайте по перевозке в Армению...
108,Внутрикорпоративные термины - Продуктовая книга,"Доставка товара в определенные дату и время, в..."
109,Концепт ролевой модели прав в новом ЛК,Права полномочий пользователя ЛК (свойства лог...
110,Программа лояльности,Регистрация в программе лояльности в офисе Ч...


Парсинг текстовой таблицы со ссылками

In [15]:
links=pd.read_csv('/content/drive/MyDrive/PEK/CMK.txt', sep='/t', header=None)
display(links) #links

<ipython-input-15-3849c0528f91>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  links=pd.read_csv('/content/drive/MyDrive/PEK/CMK.txt', sep='/t', header=None)


,0
0,"Стандарт ""Оформление выдачи груза"";http://conf..."
1,"Технологическая инструкция ""Визуальная проверк..."
2,"Технологическая инструкция ""Проверка данных ко..."
3,Технологическая инструкция «Правила подписания...
4,"Технологическая инструкция ""Архивация документ..."
...,...
107,GLOBAL;http://confluence/display/prod/GLOBAL
108,Внутрикорпоративные термины;http://confluence/...
109,Концепт ролевой модели прав в новом ЛК;http://...
110,Программа лояльности;http://confluence/pages/v...


Разделяю ссылки и названия файлов на два столбца (простой сплит по точке запятой не сработал)

In [16]:
# Split the unnamed column by the last ";"

# Function to split by the last semicolon
def split_by_last_semicolon(text):
    parts = text.rsplit(';', 1)
    return parts if len(parts) == 2 else [text, '']

# Apply the function to split the unnamed column
links_split = links.iloc[:, 0].apply(split_by_last_semicolon).apply(pd.Series)

# Rename the columns
links_split.columns = ['File', 'Link']

display(links_split)

,File,Link
0,"Стандарт ""Оформление выдачи груза""",http://confluence/pages/viewpage.action?pageId...
1,"Технологическая инструкция ""Визуальная проверк...",http://confluence/pages/viewpage.action?pageId...
2,"Технологическая инструкция ""Проверка данных ко...",http://confluence/pages/viewpage.action?pageId...
3,Технологическая инструкция «Правила подписания...,http://confluence/pages/viewpage.action?pageId...
4,"Технологическая инструкция ""Архивация документ...",http://confluence/pages/viewpage.action?pageId...
...,...,...
107,GLOBAL,http://confluence/display/prod/GLOBAL
108,Внутрикорпоративные термины,http://confluence/pages/viewpage.action?pageId...
109,Концепт ролевой модели прав в новом ЛК,http://confluence/pages/viewpage.action?pageId...
110,Программа лояльности,http://confluence/pages/viewpage.action?pageId...


Функция, которая мэтчит название файла из большой таблицы с данными и из маленькой таблицы со ссылками

In [17]:
def get_best_match(query, choices, threshold=80):
    result = process.extractOne(query, choices)
    if result:
        best_match = result[0]
        score = result[1]
        if score >= threshold:
            return best_match
    return None

In [18]:
links_split['Best_Match'] = links_split['File'].apply(lambda x: get_best_match(x, combined_paragraphs_df['Query']))

In [19]:
display(links_split)

,File,Link,Best_Match
0,"Стандарт ""Оформление выдачи груза""",http://confluence/pages/viewpage.action?pageId...,Стандарт _Оформление выдачи груза_
1,"Технологическая инструкция ""Визуальная проверк...",http://confluence/pages/viewpage.action?pageId...,Технологическая инструкция _Визуальная проверк...
2,"Технологическая инструкция ""Проверка данных ко...",http://confluence/pages/viewpage.action?pageId...,Технологическая инструкция _Проверка данных ко...
3,Технологическая инструкция «Правила подписания...,http://confluence/pages/viewpage.action?pageId...,Технологическая инструкция «Правила подписания...
4,"Технологическая инструкция ""Архивация документ...",http://confluence/pages/viewpage.action?pageId...,Технологическая инструкция _Архивация документ...
...,...,...,...
107,GLOBAL,http://confluence/display/prod/GLOBAL,GLOBAL - Продуктовая книга
108,Внутрикорпоративные термины,http://confluence/pages/viewpage.action?pageId...,Внутрикорпоративные термины - Продуктовая книга
109,Концепт ролевой модели прав в новом ЛК,http://confluence/pages/viewpage.action?pageId...,Концепт ролевой модели прав в новом ЛК
110,Программа лояльности,http://confluence/pages/viewpage.action?pageId...,Программа лояльности


Теперь можно присоединить к основной таблице поле со ссылками

In [21]:
merged_cmk_df=[]
# Create a function to merge based on the custom condition
def custom_merge(df1, df2):
    merged_data = []
    for _, row1 in df1.iterrows():
        query = row1['Query']
        matching_link = None
        for _, row2 in df2.iterrows():
            if query.startswith(row2['Best_Match']):
                matching_link = row2['Link']
                break
        merged_data.append({
            'Query': row1['Query'],
            'Answer': row1['Answer'],
            'Link': matching_link
        })
    return pd.DataFrame(merged_data)

# Merge DataFrames using the custom function
merged_cmk_df = custom_merge(combined_paragraphs_df, links_split)

In [22]:
merged_cmk_df['Department'] = 'CMK'

Выгрузка цсв

In [23]:
merged_cmk_df.to_csv('/content/drive/MyDrive/merged_cmk_df.csv', index=False)